In [3]:
import pandas as pd

In [5]:
import h5py
import pywt
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

import get_ecg as ge
import read_ecg as re
#import features as ft


## The prediction data frame

In [6]:
df = pd.read_csv("./data/annotations/dnn.csv") #df as in Data Frame
print(df)

     Unnamed: 0  1dAVb  RBBB  LBBB  SB  AF  ST
0             0      0     0     0   0   0   0
1             1      0     0     1   0   0   0
2             2      0     0     0   0   0   0
3             3      0     0     0   0   0   0
4             4      0     0     0   0   0   0
..          ...    ...   ...   ...  ..  ..  ..
822         822      0     0     0   0   0   0
823         823      0     0     0   0   0   0
824         824      0     0     0   0   0   0
825         825      0     0     0   0   0   0
826         826      1     0     0   0   0   0

[827 rows x 7 columns]


In [7]:
df['SB'].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## The raw ECGs

In [93]:
ge.get_ecg()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [94]:
f = h5py.File("./data/ecg_tracings.hdf5", "r")
table_ecg = np.array(f['tracings'])

In [95]:
np.shape(table_ecg)

(827, 4096, 12)

In [96]:
list_ecg_0lead = table_ecg[:,:,0]
list_ecg_6lead = table_ecg[:,:,6]
list_ecg_7lead = table_ecg[:,:,7]
list_ecg_8lead = table_ecg[:,:,8]
list_ecg_9lead = table_ecg[:,:,9]
list_ecg_10lead = table_ecg[:,:,10]
list_ecg_11lead = table_ecg[:,:,11]

list_ecg_6lead

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.94442214, 1.94010903,
        1.94228494],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [97]:
"""For some ecg tracings, the lead no6 is completely blank"""

for k in range(323):
    print(k)
    print(list_ecg_6lead[k])
    print(re.delete_zeros(list_ecg_6lead[k]))
#problem with 323
for k in range(324,827):
    print(k)
    print(list_ecg_6lead[k])
    print(re.delete_zeros(list_ecg_6lead[k]))


0
[0. 0. 0. ... 0. 0. 0.]
[-0.07817473 -0.06908323 -0.05304126 ... -0.07238022 -0.08794657
 -0.02140678]
1
[0.         0.         0.         ... 1.94442214 1.94010903 1.94228494]
[-5.17453238e-03  4.42719787e-03 -1.16215413e-17 ...  1.94442214e+00
  1.94010903e+00  1.94228494e+00]
2
[0. 0. 0. ... 0. 0. 0.]
[0.00977184 0.01410161 0.00275056 ... 0.02114892 0.01954368 0.00419201]
3
[0. 0. 0. ... 0. 0. 0.]
[0.04885921 0.04881544 0.02310446 ... 0.04869107 0.04885921 0.01201889]
4
[0. 0. 0. ... 0. 0. 0.]
[ 3.26779381e-02  3.47858190e-02  1.46838202e-02 ... -5.44616306e-03
  1.62784463e-19 -9.66354888e-05]
5
[0. 0. 0. ... 0. 0. 0.]
[0.09771841 0.11091459 0.10682739 ... 0.11313056 0.09771841 0.02065965]
6
[0. 0. 0. ... 0. 0. 0.]
[0.24429604 0.27187936 0.2653366  ... 0.27160416 0.24429604 0.05248796]
7
[0. 0. 0. ... 0. 0. 0.]
[0.29315524 0.35432491 0.24541923 ... 0.28868315 0.27361156 0.06000982]
8
[0. 0. 0. ... 0. 0. 0.]
[-1.26653528e-03  9.53249226e-03  4.01693226e-02 ... -1.38515294e-03
 -2.

[0.93315927 0.93573073 0.92988847 ... 0.55351435 0.54878661 0.54896694]
87
[0. 0. 0. ... 0. 0. 0.]
[0.1027003  0.1558534  0.05068243 ... 0.11020517 0.13621281 0.06239093]
88
[0. 0. 0. ... 0. 0. 0.]
[0.04885921 0.11751245 0.16547357 ... 0.03849141 0.04885921 0.01166038]
89
[0. 0. 0. ... 0. 0. 0.]
[0.09197897 0.09964231 0.08477763 ... 0.0345878  0.03875069 0.04669638]
90
[0. 0. 0. ... 0. 0. 0.]
[0.08370458 0.10869505 0.07812424 ... 0.09766851 0.1068782  0.04945808]
91
[0. 0. 0. ... 0. 0. 0.]
[-0.15634946 -0.14588912 -0.11314021 ... -0.37994345 -0.27361156
 -0.05205028]
92
[0. 0. 0. ... 0. 0. 0.]
[ 0.02931552  0.03065551 -0.00011315 ...  0.02588688  0.02931552
  0.00638261]
93
[-1.02135284 -1.02631741 -1.03221078 ... -0.68964432 -0.69811895
 -0.70155226]
[-1.02135284 -1.02631741 -1.03221078 ... -0.68964432 -0.69811895
 -0.70155226]
94
[0. 0. 0. ... 0. 0. 0.]
[0.05446247 0.0877371  0.0951173  ... 0.05106805 0.05296323 0.0249549 ]
95
[-2.84083361 -2.8264693  -2.80490946 ... -1.97311112 -1.9

183
[-0.61368187 -0.61706907 -0.61738498 ... 14.83505751 14.8477274
 14.84303046]
[-0.61368187 -0.61706907 -0.61738498 ... 14.83505751 14.8477274
 14.84303046]
184
[0. 0. 0. ... 0. 0. 0.]
[-1.14330545 -1.2101492  -0.24755239 ... -1.0616823  -1.14330545
 -0.26132947]
185
[-1.88396376 -1.87745479 -1.86739896 ... -1.43289626 -1.43294283
 -1.4310033 ]
[-1.88396376 -1.87745479 -1.86739896 ... -1.43289626 -1.43294283
 -1.4310033 ]
186
[0. 0. 0. ... 0. 0. 0.]
[-0.01954368 -0.01905413 -0.02496048 ... -0.04779734 -0.02931552
 -0.00453691]
187
[0. 0. 0. ... 0. 0. 0.]
[ 0.02931552  0.04270459  0.18529662 ... -0.08768679 -0.03908737
 -0.00548632]
188
[-1.23621978 -1.22433691 -1.22714794 ... -0.70495834 -0.70122736
 -0.69027536]
[-1.23621978 -1.22433691 -1.22714794 ... -0.70495834 -0.70122736
 -0.69027536]
189
[0. 0. 0. ... 0. 0. 0.]
[-0.07817473 -0.09478345 -0.04641968 ... -0.09654205 -0.08794657
 -0.01863653]
190
[-4.05516924 -5.59968841 -7.18758065 ... -1.0390485  -0.94513252
 -0.92262643]
[-4.0

[2.82009767 2.81791817 2.8201535  ... 0.12082445 0.12195259 0.12802191]
275
[0. 0. 0. ... 0. 0. 0.]
[ 0.09771841  0.29018137 -0.6902361  ...  0.11339124  0.09771841
  0.02051549]
276
[-1.41781929 -1.41824353 -1.42532095 ... -1.22711708 -1.22714794
 -1.22439136]
[-1.41781929 -1.41824353 -1.42532095 ... -1.22711708 -1.22714794
 -1.22439136]
277
[0. 0. 0. ... 0. 0. 0.]
[0.00977184 0.00305496 0.00024117 ... 0.02145484 0.01954368 0.00409428]
278
[0. 0. 0. ... 0. 0. 0.]
[ 0.00625193 -0.01498966  0.02408905 ...  0.03383273  0.01954368
  0.00383445]
279
[-0.39710991 -0.41851983 -0.41921201 ... -0.34096803 -0.3353696
 -0.34475426]
[-0.39710991 -0.41851983 -0.41921201 ... -0.34096803 -0.3353696
 -0.34475426]
280
[0. 0. 0. ... 0. 0. 0.]
[0.06794233 0.06872179 0.05563686 ... 0.02383086 0.01841703 0.01315661]
281
[0. 0. 0. ... 0. 0. 0.]
[ 0.05327108  0.05631627  0.05355356 ... -0.06151793 -0.06484945
 -0.0715043 ]
282
[0. 0. 0. ... 0. 0. 0.]
[0.04738599 0.05109959 0.04091274 ... 0.03433552 0.061985

[0.03625278 0.07887096 0.06135833 ... 0.01591795 0.03578075 0.02606065]
374
[2.65146833 2.64853888 2.64484664 ... 1.91263181 1.91313126 1.90499212]
[2.65146833 2.64853888 2.64484664 ... 1.91263181 1.91313126 1.90499212]
375
[0. 0. 0. ... 0. 0. 0.]
[0.14970704 0.00326363 0.01529214 ... 0.24564566 0.26909591 0.11623808]
376
[0. 0. 0. ... 0. 0. 0.]
[0.53745128 0.63332999 0.22607879 ... 0.62387603 0.5667668  0.12335509]
377
[0. 0. 0. ... 0. 0. 0.]
[-0.77197548 -0.81328055 -0.80662036 ... -0.6940774  -0.71334443
 -0.15969157]
378
[0. 0. 0. ... 0. 0. 0.]
[0.03908737 0.06411575 0.07369414 ... 0.03459958 0.03908737 0.00925213]
379
[0. 0. 0. ... 0. 0. 0.]
[ 5.41249288e-02 -1.33648305e-01  2.02213106e-01 ...  4.04931032e-02
  5.35641209e-19 -3.56555714e-03]
380
[0. 0. 0. ... 0. 0. 0.]
[0.09771841 0.11200479 0.10405139 ... 0.08128892 0.08794657 0.02063859]
381
[0. 0. 0. ... 0. 0. 0.]
[0.02931552 0.03281462 0.01955584 ... 0.04522284 0.02931552 0.00539837]
382
[0. 0. 0. ... 0. 0. 0.]
[-0.0431899  -

[-0.03442693  0.04630723 -0.60561247 ... -0.06470264 -0.05430654
 -0.02444455]
533
[0.41646118 0.43120147 0.44207812 ... 0.84397983 0.83842403 0.83072138]
[0.41646118 0.43120147 0.44207812 ... 0.84397983 0.83842403 0.83072138]
534
[0. 0. 0. ... 0. 0. 0.]
[0.07817473 0.02185637 0.07574883 ... 0.14085806 0.1172621  0.02443711]
535
[0. 0. 0. ... 0. 0. 0.]
[0.03908737 0.04856229 0.02076675 ... 0.06814542 0.04885921 0.00903938]
536
[0. 0. 0. ... 0. 0. 0.]
[0.04885921 0.04396151 0.03438047 ... 0.09585755 0.08794657 0.01884348]
537
[0.1783576  0.17104337 0.16006275 ... 0.18798264 0.18292888 0.18371164]
[0.1783576  0.17104337 0.16006275 ... 0.18798264 0.18292888 0.18371164]
538
[0. 0. 0. ... 0. 0. 0.]
[0.09771841 0.05982308 0.02149355 ... 0.09616816 0.09771841 0.02269853]
539
[0. 0. 0. ... 0. 0. 0.]
[ 9.77184146e-03  2.03344092e-02 -1.84350415e-02 ...  5.89395777e-04
  2.00967964e-19 -3.26486688e-04]
540
[0. 0. 0. ... 0. 0. 0.]
[-0.03908737 -0.00976077 -0.00088546 ... -0.0501351  -0.04885921
 

[-4.82979268 -4.82773218 -4.8247492  ... -2.12910435 -2.12654809
 -2.1136924 ]
626
[0. 0. 0. ... 0. 0. 0.]
[-0.18566499 -0.19360529 -0.15435874 ... -0.17189219 -0.17589315
 -0.04078548]
627
[-1.34022611 -1.34957061 -1.34910056 ...  1.07012719  1.06708513
  1.06849806]
[-1.34022611 -1.34957061 -1.34910056 ...  1.07012719  1.06708513
  1.06849806]
628
[0. 0. 0. ... 0. 0. 0.]
[0.08687832 0.14290922 0.16279374 ... 0.08764896 0.10900765 0.04837916]
629
[0. 0. 0. ... 0. 0. 0.]
[-0.02931552 -0.09697557 -0.14311751 ... -0.08018337 -0.06840289
 -0.01455702]
630
[0. 0. 0. ... 0. 0. 0.]
[0.08794657 0.10909926 0.01810905 ... 0.10827213 0.10749026 0.02441187]
631
[0. 0. 0. ... 0. 0. 0.]
[ 0.14657762  0.04061481 -0.01858381 ...  0.16314496  0.13680578
  0.02822517]
632
[0.39521939 0.39578027 0.3963459  ... 0.97620258 0.98324271 0.98361394]
[0.39521939 0.39578027 0.3963459  ... 0.97620258 0.98324271 0.98361394]
633
[0. 0. 0. ... 0. 0. 0.]
[-0.92832494 -1.21681751 -0.46648965 ... -0.51472455 -0.684028

723
[0. 0. 0. ... 0. 0. 0.]
[-0.04885921 -0.04412111 -0.03158231 ... -0.06200017 -0.05863105
 -0.01330047]
724
[-0.60679397 -0.61208623 -0.62500697 ... -0.68590252 -0.69360534
 -0.70441212]
[-0.60679397 -0.61208623 -0.62500697 ... -0.68590252 -0.69360534
 -0.70441212]
725
[0. 0. 0. ... 0. 0. 0.]
[-0.1172621  -0.25738211  0.35372046 ... -0.09961749 -0.09771841
 -0.02140125]
726
[0. 0. 0. ... 0. 0. 0.]
[0.09771841 0.11914925 0.12088075 ... 0.08785985 0.08794657 0.01991484]
727
[0. 0. 0. ... 0. 0. 0.]
[ 2.93155244e-02  6.20941470e-02  7.13198441e-02 ... -1.19592684e-02
 -5.47648547e-21  5.18848494e-04]
728
[-0.67967952 -0.67512411 -0.67073919 ... -2.63487967 -2.63722462
 -2.63532619]
[-0.67967952 -0.67512411 -0.67073919 ... -2.63487967 -2.63722462
 -2.63532619]
729
[0. 0. 0. ... 0. 0. 0.]
[0.10749026 0.1283965  0.13295378 ... 0.14012191 0.10749026 0.02061959]
730
[-0.74765327 -0.75774257 -0.77744768 ...  0.10157354  0.0838424
  0.06556627]
[-0.74765327 -0.75774257 -0.77744768 ...  0.10157

[0.02931552 0.02235868 0.06647935 ... 0.06307232 0.05863105 0.01284921]
813
[-2.53016305 -2.256534   -2.00459555 ... -0.6073616  -0.6021409
 -0.60111033]
[-2.53016305 -2.256534   -2.00459555 ... -0.6073616  -0.6021409
 -0.60111033]
814
[0. 0. 0. ... 0. 0. 0.]
[-0.01954368 -0.11282635  0.36847693 ... -0.01948467 -0.01954368
 -0.00473324]
815
[0. 0. 0. ... 0. 0. 0.]
[0.10749026 0.05233764 0.02169566 ... 0.10583658 0.09771841 0.02118734]
816
[0. 0. 0. ... 0. 0. 0.]
[0.06840289 0.07077237 0.07211796 ... 0.05364687 0.05863105 0.0141421 ]
817
[0. 0. 0. ... 0. 0. 0.]
[0.05863105 0.06050422 0.02593186 ... 0.05799241 0.05863105 0.01271938]
818
[-0.07369451 -0.07870772 -0.0838424  ... -0.22169958 -0.22103906
 -0.21121491]
[-0.07369451 -0.07870772 -0.0838424  ... -0.22169958 -0.22103906
 -0.21121491]
819
[0. 0. 0. ... 0. 0. 0.]
[-0.02931552 -0.04437196 -0.05565468 ... -0.01442502 -0.00977184
 -0.00083213]
820
[0. 0. 0. ... 0. 0. 0.]
[0.00977184 0.05013331 0.05906222 ... 0.03591483 0.01954368 0.00

In [98]:
"""For some ecg tracings, the lead no7 is also completely blank"""

for k in range(575):
    print(k)
    print(list_ecg_7lead[k])
    print(re.delete_zeros(list_ecg_7lead[k]))
#problem with 575
for k in range(576,784):
    print(k)
    print(list_ecg_7lead[k])
    print(re.delete_zeros(list_ecg_7lead[k]))
#problem with 784
for k in range(785,827):
    print(k)
    print(list_ecg_7lead[k])
    print(re.delete_zeros(list_ecg_7lead[k]))

0
[0. 0. 0. ... 0. 0. 0.]
[-0.31269893 -0.33781531 -0.28669829 ... -0.33232825 -0.31269893
 -0.06800327]
1
[0.         0.         0.         ... 2.81192361 2.80916154 2.80975488]
[-5.41483599e-03  4.63239575e-03 -1.21599305e-17 ...  2.81192361e+00
  2.80916154e+00  2.80975488e+00]
2
[0. 0. 0. ... 0. 0. 0.]
[-0.17589315 -0.19611192 -0.21289987 ... -0.18076757 -0.15634946
 -0.0335178 ]
3
[0. 0. 0. ... 0. 0. 0.]
[0.55699496 0.57075973 0.62003978 ... 0.55410458 0.52767944 0.11651151]
4
[0. 0. 0. ... 0. 0. 0.]
[-0.01954368 -0.03585201 -0.04150083 ... -0.01825346 -0.00977184
 -0.00129579]
5
[0. 0. 0. ... 0. 0. 0.]
[ 1.69552410e-03 -4.04566496e-03  6.43093303e-03 ...  1.73110423e-02
  6.37507528e-19 -2.25098504e-03]
6
[0. 0. 0. ... 0. 0. 0.]
[-0.03908737 -0.03671875 -0.02076746 ... -0.03066334 -0.02931552
 -0.00667847]
7
[0. 0. 0. ... 0. 0. 0.]
[-0.09771841 -0.11142441 -0.08564302 ... -0.13491457 -0.12703394
 -0.02828736]
8
[0. 0. 0. ... 0. 0. 0.]
[-0.04885921 -0.03947273 -0.05098501 ... -0.0

[-0.51810471 -0.51737489 -0.51829848 ...  0.08053684  0.09146444
  0.10266402]
85
[0. 0. 0. ... 0. 0. 0.]
[0.75243179 0.79651651 0.82011954 ... 0.80575235 0.75243179 0.16413917]
86
[1.51259393 1.5143416  1.50916322 ... 1.99598949 1.99697368 1.98952988]
[1.51259393 1.5143416  1.50916322 ... 1.99598949 1.99697368 1.98952988]
87
[0. 0. 0. ... 0. 0. 0.]
[0.08316914 0.11758914 0.09939446 ... 0.08733345 0.10933952 0.04824049]
88
[0. 0. 0. ... 0. 0. 0.]
[0.00977184 0.03546988 0.07865544 ... 0.01763966 0.00977184 0.001635  ]
89
[0. 0. 0. ... 0. 0. 0.]
[0.01069517 0.00660377 0.00138861 ... 0.12990839 0.12607791 0.13263112]
90
[0. 0. 0. ... 0. 0. 0.]
[0.0382019  0.05901191 0.00968347 ... 0.03745076 0.04295688 0.01969839]
91
[0. 0. 0. ... 0. 0. 0.]
[-0.13680578 -0.15776582 -0.08307234 ... -0.35698676 -0.23452419
 -0.04278108]
92
[0. 0. 0. ... 0. 0. 0.]
[-0.18566499 -0.20040795 -0.12771008 ... -0.18459577 -0.18566499
 -0.04217092]
93
[-0.35061368 -0.34288351 -0.34636576 ...  0.2121269   0.238805
 

[-0.33039074 -0.32679651 -0.32774757 ... -0.54670786 -0.54116459
 -0.53981706]
203
[0. 0. 0. ... 0. 0. 0.]
[-0.07817473 -0.09539345 -0.02495148 ... -0.07271695 -0.06840289
 -0.01527558]
204
[-9.07789563 -8.91064757 -8.78058583 ... -4.37540972 -4.36742684
 -4.34132522]
[-9.07789563 -8.91064757 -8.78058583 ... -4.37540972 -4.36742684
 -4.34132522]
205
[0. 0. 0. ... 0. 0. 0.]
[-0.0223328  -0.03732882 -0.0501333  ...  0.08220909  0.07741979
  0.07493136]
206
[0. 0. 0. ... 0. 0. 0.]
[0.14657762 0.17534542 0.14485418 ... 0.09147027 0.10749026 0.02621736]
207
[0. 0. 0. ... 0. 0. 0.]
[-0.01954368 -0.02039373 -0.0264491  ...  0.01882239  0.00977184
  0.00120982]
208
[2.00372546 2.00008989 1.9893515  ... 2.56530479 2.56100408 2.55902184]
[2.00372546 2.00008989 1.9893515  ... 2.56530479 2.56100408 2.55902184]
209
[0. 0. 0. ... 0. 0. 0.]
[-0.29315524 -0.33165108 -0.29171317 ... -0.33200101 -0.29315524
 -0.06235298]
210
[0. 0. 0. ... 0. 0. 0.]
[0.08394243 0.08416832 0.06619989 ... 0.01973874 0.0139

[-0.06840289 -0.06836669 -0.04743662 ... -0.09206152 -0.07817473
 -0.01636062]
348
[0. 0. 0. ... 0. 0. 0.]
[0.02931552 0.02573705 0.04759265 ... 0.02641651 0.02931552 0.00626062]
349
[0. 0. 0. ... 0. 0. 0.]
[-0.02931924 -0.01981056 -0.0086031  ... -0.05414032 -0.04773531
 -0.04407869]
350
[0. 0. 0. ... 0. 0. 0.]
[-0.1172621  -0.11743358 -0.10306735 ... -0.13706999 -0.12703394
 -0.02763142]
351
[-0.11347816 -0.11549556 -0.12195259 ... -0.15932053 -0.16006275
 -0.15570407]
[-0.11347816 -0.11549556 -0.12195259 ... -0.15932053 -0.16006275
 -0.15570407]
352
[0. 0. 0. ... 0. 0. 0.]
[-0.06840289 -0.07667874 -0.09637088 ... -0.08503661 -0.07817473
 -0.01697953]
353
[0. 0. 0. ... 0. 0. 0.]
[0.1172621  0.06818222 0.00719176 ... 0.12834657 0.12703394 0.02892536]
354
[0. 0. 0. ... 0. 0. 0.]
[-0.07817473 -0.09894273 -0.09346241 ... -0.09364053 -0.08794657
 -0.01957872]
355
[-0.29725944 -0.29103656 -0.28749353 ... -0.67316653 -0.68937052
 -0.7051254 ]
[-0.29725944 -0.29103656 -0.28749353 ... -0.6731

[0.81281916 0.81341388 0.80793593 ... 0.04582185 0.04573222 0.04557181]
485
[0. 0. 0. ... 0. 0. 0.]
[0.35000682 0.19502359 0.18053383 ... 0.41818579 0.51878002 0.24047338]
486
[0. 0. 0. ... 0. 0. 0.]
[-0.08794657 -0.03228239  0.00102096 ... -0.09171278 -0.08794657
 -0.0203041 ]
487
[0. 0. 0. ... 0. 0. 0.]
[-0.10050577 -0.15348362 -0.01725129 ... -0.14540677 -0.14940903
 -0.06290728]
488
[-1.33792788 -1.33907304 -1.33385636 ... -0.65777658 -0.66311717
 -0.66775114]
[-1.33792788 -1.33907304 -1.33385636 ... -0.65777658 -0.66311717
 -0.66775114]
489
[0. 0. 0. ... 0. 0. 0.]
[-0.45927655 -0.46567307 -0.53082765 ... -0.45955152 -0.43973287
 -0.09707577]
490
[0. 0. 0. ... 0. 0. 0.]
[-0.11056923  0.17857238  0.45942017 ... -0.07407939 -0.07993261
 -0.03906532]
491
[0. 0. 0. ... 0. 0. 0.]
[ 0.00591897  0.04671445 -0.00279784 ... -0.03216438 -0.02931552
 -0.00612128]
492
[0. 0. 0. ... 0. 0. 0.]
[-0.02931552 -0.04128724  0.04723237 ... -0.03718799 -0.02931552
 -0.00622002]
493
[0. 0. 0. ... 0. 0. 

[0.62740459 0.6243241  0.62500697 ... 0.43962267 0.44970015 0.46143597]
[0.62740459 0.6243241  0.62500697 ... 0.43962267 0.44970015 0.46143597]
615
[0. 0. 0. ... 0. 0. 0.]
[ 0.00128233 -0.01420495 -0.00377461 ... -0.00035819  0.00230693
  0.00561586]
616
[0. 0. 0. ... 0. 0. 0.]
[-0.13680578 -0.23283751  0.14522657 ... -0.19382657 -0.15634946
 -0.03278172]
617
[-5.93601471 -5.93969257 -5.93756685 ...  0.52072     0.53354257
  0.54537829]
[-5.93601471 -5.93969257 -5.93756685 ...  0.52072     0.53354257
  0.54537829]
618
[-0.39027371 -0.40060342 -0.41158999 ... -1.50907293 -1.49391917
 -1.47301323]
[-0.39027371 -0.40060342 -0.41158999 ... -1.50907293 -1.49391917
 -1.47301323]
619
[0. 0. 0. ... 0. 0. 0.]
[-0.12703394 -0.13185623 -0.1528642  ... -0.14316751 -0.12703394
 -0.02760836]
620
[0. 0. 0. ... 0. 0. 0.]
[-0.00069929  0.00113286 -0.00158853 ...  0.00535264  0.0051109
  0.00071645]
621
[0. 0. 0. ... 0. 0. 0.]
[-0.63516969 -0.93813041 -1.16195168 ...  0.19956413 -0.15634946
 -0.06210352

[-2.27741611 -2.27703597 -2.27898888 ... -1.16794024 -1.17379362
 -1.1779758 ]
720
[1.94661323 1.95372678 1.96648543 ... 0.07361943 0.06097629 0.06605139]
[1.94661323 1.95372678 1.96648543 ... 0.07361943 0.06097629 0.06605139]
721
[0. 0. 0. ... 0. 0. 0.]
[-0.06840289 -0.04867718 -0.07803531 ... -0.06852244 -0.05863105
 -0.0128454 ]
722
[0. 0. 0. ... 0. 0. 0.]
[0.04885921 0.04603531 0.02947991 ... 0.05205429 0.04885921 0.01098737]
723
[0. 0. 0. ... 0. 0. 0.]
[-0.1661213  -0.18367567 -0.16925325 ... -0.18016355 -0.17589315
 -0.03992218]
724
[0.840073   0.83651569 0.83080193 ... 0.86113205 0.86129017 0.85643736]
[0.840073   0.83651569 0.83080193 ... 0.86113205 0.86129017 0.85643736]
725
[0. 0. 0. ... 0. 0. 0.]
[-0.13680578 -0.15208589 -0.0729682  ... -0.16445918 -0.13680578
 -0.02750218]
726
[0. 0. 0. ... 0. 0. 0.]
[0.02931552 0.04541894 0.05737974 ... 0.04484695 0.03908737 0.00762646]
727
[0. 0. 0. ... 0. 0. 0.]
[-0.00977184 -0.01142559 -0.00479348 ... -0.0609076  -0.02931552
 -0.0054382

[ 0.00067539  0.02968152 -0.00211103 ... -0.02981318 -0.01954368
 -0.00359786]
812
[0. 0. 0. ... 0. 0. 0.]
[-1.95436829e-02 -2.15289816e-02 -1.72667666e-02 ... -8.46902569e-03
 -2.18342857e-19  1.03114694e-03]
813
[-2.3462508  -2.04158798 -1.76069047 ... -0.15118497 -0.1448187
 -0.13889852]
[-2.3462508  -2.04158798 -1.76069047 ... -0.15118497 -0.1448187
 -0.13889852]
814
[0. 0. 0. ... 0. 0. 0.]
[-1.67098489 -2.06616328 -0.81977638 ... -1.62999832 -1.56349463
 -0.3453247 ]
815
[0. 0. 0. ... 0. 0. 0.]
[0.06840289 0.05109641 0.02977677 ... 0.0824623  0.06840289 0.01376541]
816
[0. 0. 0. ... 0. 0. 0.]
[-0.1172621  -0.12175284 -0.08951978 ... -0.10042942 -0.10749026
 -0.02508198]
817
[0. 0. 0. ... 0. 0. 0.]
[0.33224261 0.33057492 0.25147449 ... 0.35982625 0.34201445 0.07563438]
818
[2.84447537 2.83889467 2.83539755 ... 0.49155774 0.48018832 0.47242036]
[2.84447537 2.83889467 2.83539755 ... 0.49155774 0.48018832 0.47242036]
819
[0. 0. 0. ... 0. 0. 0.]
[-0.1661213  -0.16081263 -0.17060994 ...

In [99]:
"""For some ecg tracings, the lead no8 is also completely blank"""

for k in range(177):
    print(k)
    print(list_ecg_8lead[k])
    print(re.delete_zeros(list_ecg_8lead[k]))
#problem with 177
for k in range(178,537):
    print(k)
    print(list_ecg_8lead[k])
    print(re.delete_zeros(list_ecg_8lead[k]))
#problem with 537
for k in range(538,827):
    print(k)
    print(list_ecg_8lead[k])
    print(re.delete_zeros(list_ecg_8lead[k]))

0
[0. 0. 0. ... 0. 0. 0.]
[-0.32247077 -0.36504889 -0.31484949 ... -0.35896117 -0.32247077
 -0.06837248]
1
[0.         0.         0.         ... 1.26077619 1.25350277 1.25819012]
[-3.68465170e-03  3.14636174e-03 -8.24437610e-18 ...  1.26077619e+00
  1.25350277e+00  1.25819012e+00]
2
[0. 0. 0. ... 0. 0. 0.]
[-0.17589315 -0.17420621 -0.17119712 ... -0.18967232 -0.17589315
 -0.03835176]
3
[0. 0. 0. ... 0. 0. 0.]
[0.55699496 0.57614198 0.63452065 ... 0.54294904 0.5179076  0.11473464]
4
[0. 0. 0. ... 0. 0. 0.]
[-0.03908737 -0.04946198 -0.04831662 ... -0.02854125 -0.01954368
 -0.00341889]
5
[0. 0. 0. ... 0. 0. 0.]
[-0.00293143  0.00703274 -0.01127418 ...  0.01449043  0.01954368
  0.00460681]
6
[0. 0. 0. ... 0. 0. 0.]
[0.1172621  0.10985684 0.09174207 ... 0.12570881 0.12703394 0.02774633]
7
[0. 0. 0. ... 0. 0. 0.]
[0.24429604 0.2580482  0.20729319 ... 0.24092958 0.23452419 0.05156682]
8
[0. 0. 0. ... 0. 0. 0.]
[-0.05863105 -0.04096354 -0.01765499 ... -0.05878829 -0.06840289
 -0.01642334]
9
[0

[0.57848299 0.55332753 0.52592054 ... 0.14458847 0.1448187  0.14807541]
78
[-1.15439231 -1.21693942 -1.28050204 ... -6.22871318 -6.12811776
 -5.92002668]
[-1.15439231 -1.21693942 -1.28050204 ... -6.22871318 -6.12811776
 -5.92002668]
79
[0. 0. 0. ... 0. 0. 0.]
[-0.13594751 -0.16862622 -0.16356421 ... -0.14928798 -0.17598133
 -0.08309104]
80
[0. 0. 0. ... 0. 0. 0.]
[-0.07817473 -0.09041788 -0.0897009  ... -0.0717905  -0.06840289
 -0.01574042]
81
[0. 0. 0. ... 0. 0. 0.]
[-0.23452419 -0.24329027 -0.20928277 ... -0.26325401 -0.24429604
 -0.05332897]
82
[0. 0. 0. ... 0. 0. 0.]
[ 0.11623459 -1.47552821 -3.0878792  ... -0.18097121 -0.20192455
 -0.08771109]
83
[0. 0. 0. ... 0. 0. 0.]
[0.57157823 0.83296161 0.70668431 ... 0.00887158 0.00977184 0.00285999]
84
[-0.80789619 -0.80689905 -0.80793593 ... -0.30411296 -0.29725944
 -0.28394727]
[-0.80789619 -0.80689905 -0.80793593 ... -0.30411296 -0.29725944
 -0.28394727]
85
[0. 0. 0. ... 0. 0. 0.]
[0.99672783 1.03657869 1.04575611 ... 1.0497356  0.98695

[-0.12481279 -0.12341323 -0.13719665 ... -0.01291284 -0.00762204
 -0.00160961]
165
[0. 0. 0. ... 0. 0. 0.]
[0.42996102 0.5814851  0.54718432 ... 0.4612747  0.42018918 0.09077131]
166
[0. 0. 0. ... 0. 0. 0.]
[-0.04630073 -0.02278802 -0.03415192 ... -0.01510958 -0.04283329
 -0.02580828]
167
[0. 0. 0. ... 0. 0. 0.]
[-0.10749026 -0.14796643 -0.13846281 ... -0.08161007 -0.08794657
 -0.02004487]
168
[0. 0. 0. ... 0. 0. 0.]
[-0.13630953 -0.16790889 -0.16672571 ... -0.15416272 -0.17509774
 -0.07741118]
169
[0. 0. 0. ... 0. 0. 0.]
[0.06840289 0.05985638 0.03582607 ... 0.08254877 0.08794657 0.01954813]
170
[-0.11886116 -0.11188467 -0.10670851 ... -0.6166417  -0.60976292
 -0.60734487]
[-0.11886116 -0.11188467 -0.10670851 ... -0.6166417  -0.60976292
 -0.60734487]
171
[0.19295767 0.17631559 0.16006275 ... 1.21428691 1.20428179 1.20322801]
[0.19295767 0.17631559 0.16006275 ... 1.21428691 1.20428179 1.20322801]
172
[0. 0. 0. ... 0. 0. 0.]
[0.34201445 0.35532739 0.22577187 ... 0.45518728 0.39087366 0.

[-0.14657762 -0.14536824 -0.19190747 ... -0.12975676 -0.12703394
 -0.02843787]
260
[-1.18547368 -1.53470584 -1.81404464 ... -2.02361731 -2.11892607
 -2.13312079]
[-1.18547368 -1.53470584 -1.81404464 ... -2.02361731 -2.11892607
 -2.13312079]
261
[0. 0. 0. ... 0. 0. 0.]
[-2.42341668 -2.08379729 -1.45668371 ... -4.10876988 -3.205164
 -0.65519745]
262
[-0.59236399 -0.59403966 -0.59451887 ... -2.44014384 -2.44667356
 -2.44420362]
[-0.59236399 -0.59403966 -0.59451887 ... -2.44014384 -2.44667356
 -2.44420362]
263
[-0.09146444 -0.08665118 -0.07498314 ... -0.25756401 -0.26253047
 -0.26858403]
[-0.09146444 -0.08665118 -0.07498314 ... -0.25756401 -0.26253047
 -0.26858403]
264
[0.38144168 0.39297526 0.41158999 ... 0.64839924 0.64787312 0.64703924]
[0.38144168 0.39297526 0.41158999 ... 0.64839924 0.64787312 0.64703924]
265
[-0.80171446 -0.80163692 -0.80793593 ... -1.09633666 -1.0899512
 -1.08770704]
[-0.80171446 -0.80163692 -0.80793593 ... -1.09633666 -1.0899512
 -1.08770704]
266
[ 0.02794082  0.03

[0.19543683 0.1064837  0.01351787 ... 0.21942041 0.21498051 0.04746012]
354
[0. 0. 0. ... 0. 0. 0.]
[-0.05863105 -0.08176795 -0.07018235 ... -0.05182108 -0.04885921
 -0.01090546]
355
[ 0.37347979  0.37981288  0.37952779 ... -0.16694691 -0.19434804
 -0.22137234]
[ 0.37347979  0.37981288  0.37952779 ... -0.16694691 -0.19434804
 -0.22137234]
356
[0. 0. 0. ... 0. 0. 0.]
[0.20520867 0.22795848 0.20395715 ... 0.24562178 0.21498051 0.04603088]
357
[-0.80206485 -0.79815693 -0.79269181 ... -0.52950976 -0.52592054
 -0.52145445]
[-0.80206485 -0.79815693 -0.79269181 ... -0.52950976 -0.52592054
 -0.52145445]
358
[-1.73782425 -1.7176402  -1.70709823 ... -1.84871976 -1.85094493
 -1.85154691]
[-1.73782425 -1.7176402  -1.70709823 ... -1.84871976 -1.85094493
 -1.85154691]
359
[0. 0. 0. ... 0. 0. 0.]
[-0.17589315 -0.17239775 -0.14507504 ... -0.15871457 -0.1661213
 -0.03743246]
360
[0. 0. 0. ... 0. 0. 0.]
[-0.07817473 -0.01060417  0.0506539  ... -0.09124515 -0.07817473
 -0.01661679]
361
[0. 0. 0. ... 0. 0

[-1.40070607e-05  3.19594874e-05 -6.09602716e-05  1.05131779e-04
 -1.70490491e-04  2.67395905e-04 -4.17691985e-04  6.70581308e-04
 -1.26928531e-03  5.32414126e-03  2.75544215e-03 -1.15811290e-03
  8.03685056e-04 -7.28604544e-04  8.14596568e-04 -1.17599111e-03
  9.29725168e-03  5.28342756e-03 -1.20915790e-03  5.78631736e-04
 -2.76323875e-04 -3.24171287e-05  1.14138081e-03  1.09582215e-02
  1.14138081e-03 -3.24171287e-05 -2.76323875e-04  5.78631736e-04
 -1.20915790e-03  5.28342756e-03  9.29725168e-03 -1.17599111e-03
  8.14596568e-04 -7.38518134e-04  8.14596568e-04 -1.17599111e-03
  9.29725168e-03  5.28342756e-03 -1.20915790e-03  5.68718146e-04
 -2.51405302e-04 -9.21684181e-05  1.25470607e-03  1.07823655e-02
  1.42457532e-03 -4.30237814e-04  3.81869079e-04 -5.17300667e-04
  1.62280936e-03  1.02953349e-02  9.62359225e-03  6.74121434e-03
 -1.23001835e-02 -1.42314748e-02 -2.52474496e-02 -4.64707314e-02
 -4.06785318e-02 -3.88200646e-02 -3.13357595e-02 -8.95075718e-03
 -5.28604105e-03 -1.63347

574
[0. 0. 0. ... 0. 0. 0.]
[0.10749026 0.1310233  0.0975453  ... 0.14032074 0.1172621  0.0247223 ]
575
[1.00421883 1.00612022 1.00610879 ... 0.5036606  0.51067642 0.52271861]
[1.00421883 1.00612022 1.00610879 ... 0.5036606  0.51067642 0.52271861]
576
[0. 0. 0. ... 0. 0. 0.]
[-0.05653901 -0.0806502  -0.02174585 ... -0.06652916 -0.07863116
 -0.03310341]
577
[1.16634939 1.19710593 1.21952584 ... 0.46085247 0.57927479 0.73736275]
[1.16634939 1.19710593 1.21952584 ... 0.46085247 0.57927479 0.73736275]
578
[1.26679317 1.26739787 1.28050204 ... 1.01999645 1.00610879 1.02846254]
[1.26679317 1.26739787 1.28050204 ... 1.01999645 1.00610879 1.02846254]
579
[0. 0. 0. ... 0. 0. 0.]
[0.02931552 0.01579398 0.00224122 ... 0.0163228  0.02931552 0.00809435]
580
[0. 0. 0. ... 0. 0. 0.]
[ 0.02931552  0.01921239 -0.00066162 ...  0.04712739  0.04885921
  0.01056997]
581
[ 0.22423038  0.24171893  0.25914924 ... -0.12579646 -0.22103906
 -0.32220214]
[ 0.22423038  0.24171893  0.25914924 ... -0.12579646 -0.221

 -0.05193867]
718
[-1.61802616 -1.61261803 -1.60824968 ... -1.15783962 -1.13568343
 -1.11013092]
[-1.61802616 -1.61261803 -1.60824968 ... -1.15783962 -1.13568343
 -1.11013092]
719
[-2.70924918 -2.70193726 -2.70582284 ... -1.53187568 -1.53202944
 -1.53035505]
[-2.70924918 -2.70193726 -2.70582284 ... -1.53187568 -1.53202944
 -1.53035505]
720
[2.60184097 2.59714642 2.60673652 ... 1.39255287 1.38721068 1.39182031]
[2.60184097 2.59714642 2.60673652 ... 1.39255287 1.38721068 1.39182031]
721
[0. 0. 0. ... 0. 0. 0.]
[-0.13680578 -0.12261871 -0.14121464 ... -0.16735083 -0.15634946
 -0.03469341]
722
[0. 0. 0. ... 0. 0. 0.]
[-0.01954368 -0.00890984 -0.02661281 ... -0.01841432 -0.00977184
 -0.00138056]
723
[0. 0. 0. ... 0. 0. 0.]
[-0.07817473 -0.0732131  -0.07086673 ... -0.07220254 -0.06840289
 -0.01543694]
724
[ 0.0626831   0.05865462  0.04573222 ... -0.30052446 -0.29725944
 -0.29254261]
[ 0.0626831   0.05865462  0.04573222 ... -0.30052446 -0.29725944
 -0.29254261]
725
[0. 0. 0. ... 0. 0. 0.]
[-0

In [100]:
"""For some ecg tracings, the lead no9 is also completely blank"""

for k in range(251):
    print(k)
    print(list_ecg_9lead[k])
    print(re.delete_zeros(list_ecg_9lead[k]))
#problem with 251
for k in range(252,827):
    print(k)
    print(list_ecg_9lead[k])
    print(re.delete_zeros(list_ecg_9lead[k]))

0
[0. 0. 0. ... 0. 0. 0.]
[-0.42996102 -0.44988981 -0.40068841 ... -0.44247333 -0.41041734
 -0.08961817]
1
[0.         0.         0.         ... 1.20552059 1.20201614 1.20299696]
[-2.49915510e-03  2.13405408e-03 -5.58832844e-18 ...  1.20552059e+00
  1.20201614e+00  1.20299696e+00]
2
[0. 0. 0. ... 0. 0. 0.]
[-0.50813576 -0.65209317 -0.20209543 ... -0.55308606 -0.50813576
 -0.11095142]
3
[0. 0. 0. ... 0. 0. 0.]
[0.50813576 0.66579579 0.14616707 ... 0.49147962 0.46904839 0.10380411]
4
[0. 0. 0. ... 0. 0. 0.]
[-0.02931552 -0.05492419 -0.05941315 ... -0.02163663 -0.01954368
 -0.0041385 ]
5
[0. 0. 0. ... 0. 0. 0.]
[-0.02931552 -0.02803924 -0.0420518  ... -0.01092833 -0.00977184
 -0.00203196]
6
[0. 0. 0. ... 0. 0. 0.]
[ 0.00713751 -0.01421868  0.01552425 ... -0.02705012 -0.00977184
 -0.0001512 ]
7
[0. 0. 0. ... 0. 0. 0.]
[0.22475235 0.22280894 0.22410863 ... 0.20159446 0.20520867 0.04602471]
8
[0. 0. 0. ... 0. 0. 0.]
[-0.08794657 -0.07800888 -0.06268861 ... -0.08722363 -0.08794657
 -0.0197678

[-0.15634946 -0.14489582 -0.10390131 ... -0.17092004 -0.15634946
 -0.03326008]
129
[0. 0. 0. ... 0. 0. 0.]
[ 0.01954368 -0.06375167 -0.09922853 ...  0.03129432  0.01954368
  0.0025313 ]
130
[0. 0. 0. ... 0. 0. 0.]
[ 4.12246575e-02 -4.29184105e-02  1.06357948e-01 ... -1.32544206e-02
 -8.32649410e-20  6.95324915e-04]
131
[-0.70793864 -0.70785588 -0.70122736 ... -1.44500819 -1.44056485
 -1.43883005]
[-0.70793864 -0.70785588 -0.70122736 ... -1.44500819 -1.44056485
 -1.43883005]
132
[0. 0. 0. ... 0. 0. 0.]
[0.20554125 0.32311752 0.2390756  ... 0.08585215 0.1549912  0.13205961]
133
[0. 0. 0. ... 0. 0. 0.]
[-0.24429604 -0.22086732 -0.39986007 ... -0.23522203 -0.22475235
 -0.04972432]
134
[0. 0. 0. ... 0. 0. 0.]
[-0.00630535  0.01483462 -0.02341584 ... -0.03946531 -0.02931552
 -0.00552596]
135
[0.26098528 0.25723604 0.2515272  ... 0.62937356 0.62500697 0.61943933]
[0.26098528 0.25723604 0.2515272  ... 0.62937356 0.62500697 0.61943933]
136
[0. 0. 0. ... 0. 0. 0.]
[-0.1172621  -0.11363567 -0.105

 -0.0334818 ]
271
[0. 0. 0. ... 0. 0. 0.]
[0.02931552 0.02584798 0.00961052 ... 0.01127311 0.00977184 0.00196259]
272
[0. 0. 0. ... 0. 0. 0.]
[-0.02931552 -0.02855034 -0.03887899 ... -0.00241191 -0.00977184
 -0.00296883]
273
[0. 0. 0. ... 0. 0. 0.]
[0.13680578 0.08037895 0.02185803 ... 0.13323259 0.13680578 0.03086256]
274
[2.79206861 2.78703277 2.78966511 ... 0.19977095 0.20579499 0.21595258]
[2.79206861 2.78703277 2.78966511 ... 0.19977095 0.20579499 0.21595258]
275
[0. 0. 0. ... 0. 0. 0.]
[ 0.03908737 -0.27757316  1.28864756 ...  0.05530602  0.03908737
  0.00766065]
276
[-0.2102266  -0.19426835 -0.19055091 ...  0.33178896  0.34299166
  0.34936576]
[-0.2102266  -0.19426835 -0.19055091 ...  0.33178896  0.34299166
  0.34936576]
277
[0. 0. 0. ... 0. 0. 0.]
[-0.17589315 -0.20190279 -0.14344382 ... -0.1822303  -0.17589315
 -0.03899813]
278
[0. 0. 0. ... 0. 0. 0.]
[-0.17589315 -0.22868439 -0.08449675 ... -0.1743942  -0.17589315
 -0.03983996]
279
[-0.61491731 -0.62820545 -0.61738498 ... -0.

[-0.15634946  0.00594609  0.11338114 ... -0.19660256 -0.17589315
 -0.03790434]
419
[-3.68938956 -3.6834458  -3.67382173 ... -3.54972012 -3.55949106
 -3.55735252]
[-3.68938956 -3.6834458  -3.67382173 ... -3.54972012 -3.55949106
 -3.55735252]
420
[0. 0. 0. ... 0. 0. 0.]
[-0.17589315 -0.18291759 -0.14617561 ... -0.17700705 -0.1661213
 -0.03722584]
421
[0. 0. 0. ... 0. 0. 0.]
[0.14990563 0.16014031 0.14745227 ... 0.17035529 0.20657903 0.09210854]
422
[0. 0. 0. ... 0. 0. 0.]
[-0.14657762 -0.47849206  0.5335945  ... -0.15557613 -0.14657762
 -0.03268845]
423
[0. 0. 0. ... 0. 0. 0.]
[0.20520867 0.20038223 0.20697368 ... 0.1953432  0.19543683 0.04420847]
424
[0.69842612 0.70171691 0.70884939 ... 1.07955747 1.09757323 1.10392893]
[0.69842612 0.70171691 0.70884939 ... 1.07955747 1.09757323 1.10392893]
425
[ 0.41761749  0.43125614  0.44207812 ... -0.28122279 -0.26677128
 -0.23601319]
[ 0.41761749  0.43125614  0.44207812 ... -0.28122279 -0.26677128
 -0.23601319]
426
[0. 0. 0. ... 0. 0. 0.]
[-0.1684

[0.06840289 0.05236513 0.15058808 ... 0.05506065 0.04885921 0.00982878]
490
[0. 0. 0. ... 0. 0. 0.]
[-0.10496182  0.19934636  0.4956836  ... -0.0658499  -0.0815322
 -0.03458214]
491
[0. 0. 0. ... 0. 0. 0.]
[-0.03908737 -0.0434259   0.00048182 ... -0.07663558 -0.05863105
 -0.01197527]
492
[0. 0. 0. ... 0. 0. 0.]
[-0.19543683 -0.15873809 -0.23495676 ... -0.22783285 -0.20520867
 -0.04422788]
493
[0. 0. 0. ... 0. 0. 0.]
[0.03908737 0.04478476 0.06789219 ... 0.04818001 0.03908737 0.0082637 ]
494
[0. 0. 0. ... 0. 0. 0.]
[-0.07487513 -0.09298942 -0.10488984 ... -0.10660087 -0.10247349
 -0.0484055 ]
495
[0. 0. 0. ... 0. 0. 0.]
[ 0.01281857  0.0033079  -0.01325827 ...  0.07222621  0.05087513
  0.02911099]
496
[0. 0. 0. ... 0. 0. 0.]
[-0.01954368 -0.63656765 -0.96261079 ... -0.033076   -0.02931552
 -0.00583727]
497
[0. 0. 0. ... 0. 0. 0.]
[-0.05863105 -0.00185337 -0.01083917 ... -0.12643034 -0.09771841
 -0.01976545]
498
[-9.27175383 -9.28782458 -9.30650641 ...  0.38832227  0.38872384
  0.3832458

[0.40809138 0.42312619 0.43445606 ... 1.17915382 1.18141565 1.18454797]
583
[0. 0. 0. ... 0. 0. 0.]
[1.99345566 1.81441799 0.4893231  ... 2.48805123 2.32569827 0.50504114]
584
[-0.29951922 -0.30656977 -0.30488146 ... -0.28054968 -0.28201535
 -0.27880962]
[-0.29951922 -0.30656977 -0.30488146 ... -0.28054968 -0.28201535
 -0.27880962]
585
[0. 0. 0. ... 0. 0. 0.]
[0.20641964 0.29547086 0.23649052 ... 0.00646134 0.01954368 0.00559751]
586
[0.68103869 0.69533266 0.72409344 ... 0.33333333 0.31250349 0.30822538]
[0.68103869 0.69533266 0.72409344 ... 0.33333333 0.31250349 0.30822538]
587
[0. 0. 0. ... 0. 0. 0.]
[ 0.86969389  0.17241899 -0.28976432 ...  1.83471761  1.52440727
  0.31512502]
588
[0. 0. 0. ... 0. 0. 0.]
[-0.06840289 -0.07974817  0.00326524 ... -0.09394041 -0.08794657
 -0.01946695]
589
[0. 0. 0. ... 0. 0. 0.]
[-0.02931552 -0.10799661 -0.13765201 ... -0.04715715 -0.02931552
 -0.00465493]
590
[0. 0. 0. ... 0. 0. 0.]
[-0.59608233  0.22619723  0.71230219 ... -1.07249136 -0.81106284
 -0.

[0.33224261 0.30685321 0.39624552 ... 0.38834566 0.34201445 0.07324055]
681
[0. 0. 0. ... 0. 0. 0.]
[-0.12703394 -0.12168445 -0.08668063 ... -0.19670461 -0.1661213
 -0.03559131]
682
[0. 0. 0. ... 0. 0. 0.]
[1.79801883 2.18104974 2.66772778 ... 0.98461726 1.23125202 0.29612681]
683
[0. 0. 0. ... 0. 0. 0.]
[-0.1172621  -0.21343457 -0.21712346 ... -0.11559281 -0.10749026
 -0.02360839]
684
[0.23709796 0.23433035 0.2286611  ... 0.48040003 0.47256626 0.46085783]
[0.23709796 0.23433035 0.2286611  ... 0.48040003 0.47256626 0.46085783]
685
[0. 0. 0. ... 0. 0. 0.]
[-0.23419205 -0.32858115 -0.30381997 ... -0.26933509 -0.30413699
 -0.13627663]
686
[-1.7726922  -1.77530849 -1.77593452 ... -0.07658558 -0.07622037
 -0.07041285]
[-1.7726922  -1.77530849 -1.77593452 ... -0.07658558 -0.07622037
 -0.07041285]
687
[0. 0. 0. ... 0. 0. 0.]
[-0.10749026 -0.10193593 -0.06455099 ... -0.1410484  -0.1172621
 -0.02429205]
688
[0. 0. 0. ... 0. 0. 0.]
[-0.05863105 -0.06599283 -0.04828515 ... -0.05095135 -0.05863105

[-0.12703394 -0.13929188 -0.11247898 ... -0.12530486 -0.1172621
 -0.02594891]
763
[0. 0. 0. ... 0. 0. 0.]
[-0.25406788 -0.26931993 -0.21362273 ... -0.23679662 -0.23452419
 -0.05306846]
764
[ 0.30175527  0.30335537  0.29725944 ... -0.06645023 -0.06097629
 -0.06756908]
[ 0.30175527  0.30335537  0.29725944 ... -0.06645023 -0.06097629
 -0.06756908]
765
[0. 0. 0. ... 0. 0. 0.]
[-0.07817473 -0.02441595  0.28601486 ... -0.167569   -0.13680578
 -0.02859743]
766
[0. 0. 0. ... 0. 0. 0.]
[-0.06840289 -0.06247946 -0.02802517 ... -0.07119246 -0.05863105
 -0.01184823]
767
[0.07647655 0.07621477 0.07622037 ... 0.46775567 0.41158999 0.35417857]
[0.07647655 0.07621477 0.07622037 ... 0.46775567 0.41158999 0.35417857]
768
[0. 0. 0. ... 0. 0. 0.]
[0.03908737 0.03122098 0.06874991 ... 0.05417955 0.04885921 0.01007976]
769
[0. 0. 0. ... 0. 0. 0.]
[-6.11108668e-03 -1.68801611e-02 -1.83797307e-02 ... -8.04604220e-03
 -2.21315782e-19  9.87073415e-04]
770
[0.11192673 0.17144262 0.13719665 ... 0.50796237 0.55640

In [101]:
"""For some ecg tracings, the lead no10 is also completely blank"""

for k in range(269):
    print(k)
    print(list_ecg_10lead[k])
    print(re.delete_zeros(list_ecg_10lead[k]))
#problem with 269
for k in range(270,542):
    print(k)
    print(list_ecg_10lead[k])
    print(re.delete_zeros(list_ecg_10lead[k]))
#problem with 542
for k in range(543,601):
    print(k)
    print(list_ecg_10lead[k])
    print(re.delete_zeros(list_ecg_10lead[k]))
#problem with 601
for k in range(602,790):
    print(k)
    print(list_ecg_10lead[k])
    print(re.delete_zeros(list_ecg_10lead[k]))
#problem with 790
for k in range(791,827):
    print(k)
    print(list_ecg_10lead[k])
    print(re.delete_zeros(list_ecg_10lead[k]))

0
[0. 0. 0. ... 0. 0. 0.]
[-0.18566499 -0.20376505 -0.16162514 ... -0.20419586 -0.17589315
 -0.03729893]
1
[0.         0.         0.         ... 1.66197859 1.65950262 1.65952942]
[-2.03456839e-03  1.74595062e-03 -4.56478913e-18 ...  1.66197859e+00
  1.65950262e+00  1.65952942e+00]
2
[0. 0. 0. ... 0. 0. 0.]
[-0.14657762 -0.18799881 -0.05118165 ... -0.15757741 -0.14657762
 -0.03203265]
3
[0. 0. 0. ... 0. 0. 0.]
[0.34201445 0.4385934  0.06538993 ... 0.28911745 0.30292709 0.06969962]
4
[0. 0. 0. ... 0. 0. 0.]
[-9.77184146e-03 -6.44872097e-02 -8.78305994e-02 ... -3.76218212e-04
 -2.04395362e-19  1.86032066e-04]
5
[0. 0. 0. ... 0. 0. 0.]
[-2.93155244e-02 -9.33346474e-03 -3.24156014e-02 ...  8.98124332e-03
  3.97494933e-19 -1.34064999e-03]
6
[0. 0. 0. ... 0. 0. 0.]
[0.08794657 0.09031749 0.09599738 ... 0.08941504 0.08794657 0.01916504]
7
[0. 0. 0. ... 0. 0. 0.]
[0.22475235 0.22575332 0.21471762 ... 0.20671978 0.21498051 0.04799255]
8
[0. 0. 0. ... 0. 0. 0.]
[-0.09771841 -0.09502583 -0.0830099

[-0.13680578 -0.12549497 -0.12107377 ... -0.16441815 -0.14657762
 -0.03180953]
135
[-0.34870961 -0.35158477 -0.35823571 ...  0.01265602  0.00762204
  0.00147067]
[-0.34870961 -0.35158477 -0.35823571 ...  0.01265602  0.00762204
  0.00147067]
136
[0. 0. 0. ... 0. 0. 0.]
[-1.95436829e-02 -6.99946460e-03 -2.44973903e-02 ...  5.61343578e-03
  1.43691383e-19 -2.16118963e-04]
137
[0. 0. 0. ... 0. 0. 0.]
[-0.1172621  -0.13880646 -0.14223934 ... -0.12038061 -0.12703394
 -0.02973691]
138
[-2.88119653 -2.87826938 -2.8811297  ... -3.25916966 -3.20887735
 -3.16164306]
[-2.88119653 -2.87826938 -2.8811297  ... -3.25916966 -3.20887735
 -3.16164306]
139
[0. 0. 0. ... 0. 0. 0.]
[-0.13799688  0.23465816  0.23637336 ... -0.19519153 -0.25598036
 -0.1245679 ]
140
[ 0.36918355  0.34129738  0.31250349 ... -0.47917082 -0.43445606
 -0.39368552]
[ 0.36918355  0.34129738  0.31250349 ... -0.47917082 -0.43445606
 -0.39368552]
141
[0. 0. 0. ... 0. 0. 0.]
[-0.14657762 -0.48472496  1.19013862 ... -0.18086721 -0.156349

[-0.14657762 -0.16449199 -0.14938814 ... -0.11258857 -0.1172621
 -0.02679775]
231
[0. 0. 0. ... 0. 0. 0.]
[ 0.02931552 -0.05112958 -0.10171116 ...  0.10718101  0.06840289
  0.01269119]
232
[0. 0. 0. ... 0. 0. 0.]
[-0.00229179 -0.08776641 -0.18051336 ...  0.01068762  0.00132916
 -0.00036446]
233
[0. 0. 0. ... 0. 0. 0.]
[-0.26383972 -0.28318895 -0.28165421 ... -0.25909542 -0.25406788
 -0.05672984]
234
[-0.09145176 -0.09214661 -0.09908648 ...  0.20257645  0.19817295
  0.20975246]
[-0.09145176 -0.09214661 -0.09908648 ...  0.20257645  0.19817295
  0.20975246]
235
[ 0.3963459   0.40608553  0.41571969 ... -7.53397506 -7.5325471
 -7.54185955]
[ 0.3963459   0.40608553  0.41571969 ... -7.53397506 -7.5325471
 -7.54185955]
236
[0. 0. 0. ... 0. 0. 0.]
[1.87619356 2.12592081 2.83805184 ... 1.11565943 1.37782965 0.3283294 ]
237
[0. 0. 0. ... 0. 0. 0.]
[-0.04885921 -0.03820097 -0.14326228 ... -0.03753806 -0.02931552
 -0.00627922]
238
[0. 0. 0. ... 0. 0. 0.]
[0.90878126 0.68443632 0.00963364 ... 2.0010

[-0.04885921 -0.05603622 -0.03734209 ... -0.05199092 -0.04885921
 -0.0109685 ]
355
[-3.54424702 -3.52477365 -3.50826398 ... 15.58707078 15.56997261
 15.56554353]
[-3.54424702 -3.52477365 -3.50826398 ... 15.58707078 15.56997261
 15.56554353]
356
[0. 0. 0. ... 0. 0. 0.]
[0.26383972 0.27439282 0.23776088 ... 0.3125134  0.2833834  0.06147013]
357
[-0.75374189 -0.75396506 -0.75458161 ... -0.48828148 -0.49543237
 -0.49621095]
[-0.75374189 -0.75396506 -0.75458161 ... -0.48828148 -0.49543237
 -0.49621095]
358
[-2.05794987 -2.03950426 -2.02468474 ... -2.56793047 -2.58026967
 -2.58295641]
[-2.05794987 -2.03950426 -2.02468474 ... -2.56793047 -2.58026967
 -2.58295641]
359
[0. 0. 0. ... 0. 0. 0.]
[-0.24429604 -0.27815219 -0.25231432 ... -0.26673287 -0.25406788
 -0.05591429]
360
[0. 0. 0. ... 0. 0. 0.]
[-0.18566499 -0.13927864 -0.08169308 ... -0.19628884 -0.17589315
 -0.03804937]
361
[0. 0. 0. ... 0. 0. 0.]
[-0.16006576  0.01218101  0.22076491 ... -0.12926128 -0.15991378
 -0.0740572 ]
362
[0. 0. 0. 

[-0.99921497 -0.99934    -1.00610879 ... -0.65524137 -0.65549514
 -0.65442641]
526
[0. 0. 0. ... 0. 0. 0.]
[-0.16016334 -0.19666385 -0.17034886 ... -0.20430791 -0.22862628
 -0.10204206]
527
[-1.33088726 -1.33344029 -1.34147838 ... -0.63575721 -0.62500697
 -0.61818735]
[-1.33088726 -1.33344029 -1.34147838 ... -0.63575721 -0.62500697
 -0.61818735]
528
[-1.66261406 -1.6671919  -1.65398198 ... -0.98057813 -0.98324271
 -0.95848747]
[-1.66261406 -1.6671919  -1.65398198 ... -0.98057813 -0.98324271
 -0.95848747]
529
[0. 0. 0. ... 0. 0. 0.]
[0.32247077 0.15610143 0.02504982 ... 0.04322376 0.1172621  0.0319581 ]
530
[-0.40748174 -0.40288476 -0.40396796 ...  0.16813513  0.1676848
  0.17881542]
[-0.40748174 -0.40288476 -0.40396796 ...  0.16813513  0.1676848
  0.17881542]
531
[0. 0. 0. ... 0. 0. 0.]
[-0.12703394 -0.12778485 -0.11279298 ...  0.08128172 -0.04885921
 -0.01914817]
532
[0. 0. 0. ... 0. 0. 0.]
[0.21740172 0.27985038 0.1956877  ... 0.31917044 0.3191526  0.13959492]
533
[0.11783583 0.12424

[0.01954368 0.05865751 0.07734325 ... 0.05474255 0.03908737 0.00804308]
650
[ 0.0960289   0.10095481  0.10670851 ... -1.62814074 -1.61587186
 -1.60218117]
[ 0.0960289   0.10095481  0.10670851 ... -1.62814074 -1.61587186
 -1.60218117]
651
[0. 0. 0. ... 0. 0. 0.]
[0.022605   0.04778685 0.02135861 ... 0.04308273 0.03296464 0.01473988]
652
[0. 0. 0. ... 0. 0. 0.]
[-0.00681091 -0.09782196 -0.07446569 ...  0.0710744   0.05641923
  0.02375679]
653
[0. 0. 0. ... 0. 0. 0.]
[0.27213958 0.6678989  0.58953457 ... 0.32797803 0.33293184 0.13986542]
654
[0. 0. 0. ... 0. 0. 0.]
[-0.24429604 -0.2415338  -0.27685139 ... -0.29355516 -0.26383972
 -0.05619786]
655
[-2.74719117 -2.74751761 -2.74393326 ... -1.87293111 -1.87502099
 -1.87353872]
[-2.74719117 -2.74751761 -2.74393326 ... -1.87293111 -1.87502099
 -1.87353872]
656
[0. 0. 0. ... 0. 0. 0.]
[-0.23452419 -0.22613558 -0.22496069 ... -0.25225833 -0.23452419
 -0.05142483]
657
[0. 0. 0. ... 0. 0. 0.]
[0.1172621  0.16222261 0.17424061 ... 0.12034636 0.1074

[2.10977646 2.10018759 2.10368217 ... 0.63263185 0.63262907 0.63161713]
721
[0. 0. 0. ... 0. 0. 0.]
[-0.09771841 -0.18437052 -0.1883518  ... -0.11583199 -0.10749026
 -0.02349161]
722
[0. 0. 0. ... 0. 0. 0.]
[-0.13680578 -0.15895228 -0.1227386  ... -0.15466423 -0.13680578
 -0.02927593]
723
[0. 0. 0. ... 0. 0. 0.]
[-0.01954368 -0.02683526 -0.03287175 ... -0.01822801 -0.00977184
 -0.00136632]
724
[4.51541478 4.51724097 4.52748995 ... 0.71221662 0.71647141 0.72099575]
[4.51541478 4.51724097 4.52748995 ... 0.71221662 0.71647141 0.72099575]
725
[0. 0. 0. ... 0. 0. 0.]
[ 0.03908737 -1.5656936  -2.29719991 ...  0.05225167  0.03908737
  0.00690852]
726
[0. 0. 0. ... 0. 0. 0.]
[-0.17589315 -0.163565   -0.10779567 ... -0.19174134 -0.18566499
 -0.04145828]
727
[0. 0. 0. ... 0. 0. 0.]
[-0.15634946 -0.14617023 -0.11098291 ... -0.35607169 -0.24429604
 -0.04779196]
728
[-0.20050537 -0.19749025 -0.19817295 ...  0.35235227  0.35823571
  0.35782452]
[-0.20050537 -0.19749025 -0.19817295 ...  0.35235227  0

819
[0. 0. 0. ... 0. 0. 0.]
[-0.13680578 -0.14692608 -0.08585412 ... -0.14839649 -0.13680578
 -0.02965215]
820
[0. 0. 0. ... 0. 0. 0.]
[ 0.33224261  0.08869764 -0.07110592 ...  0.32356822  0.31269893
  0.0691007 ]
821
[0.37309965 0.37467688 0.38110182 ... 0.29955668 0.30488146 0.30507007]
[0.37309965 0.37467688 0.38110182 ... 0.29955668 0.30488146 0.30507007]
822
[0. 0. 0. ... 0. 0. 0.]
[-0.00125919 -0.08096392 -0.14927681 ...  0.00870582  0.00977184
  0.00250953]
823
[0. 0. 0. ... 0. 0. 0.]
[0.00655371 0.01354975 0.01765291 ... 0.22558398 0.2231335  0.23658139]
824
[0. 0. 0. ... 0. 0. 0.]
[-0.38110182 -0.41860617 -0.28685568 ... -0.41793671 -0.38110182
 -0.08267808]
825
[0. 0. 0. ... 0. 0. 0.]
[0.07817473 0.06893137 0.046572   ... 0.10569519 0.07817473 0.01534613]
826
[0. 0. 0. ... 0. 0. 0.]
[4.32892576 4.19371961 3.5225279  ... 4.5929926  4.42664418 0.97725129]


In [102]:
"""For some ecg tracings, the lead no11 is also completely blank"""

for k in range(321):
    print(k)
    print(list_ecg_11lead[k])
    print(re.delete_zeros(list_ecg_11lead[k]))
#problem with 321
for k in range(322,385):
    print(k)
    print(list_ecg_11lead[k])
    print(re.delete_zeros(list_ecg_11lead[k]))
#problem with 385
for k in range(386,434):
    print(k)
    print(list_ecg_11lead[k])
    print(re.delete_zeros(list_ecg_11lead[k]))
#problem with 434
for k in range(435,446):
    print(k)
    print(list_ecg_11lead[k])
    print(re.delete_zeros(list_ecg_11lead[k]))
#problem with 446
for k in range(447,827):
    print(k)
    print(list_ecg_11lead[k])
    print(re.delete_zeros(list_ecg_11lead[k]))

0
[0. 0. 0. ... 0. 0. 0.]
[-0.12703394 -0.1380552  -0.0817327  ... -0.12495896 -0.1172621
 -0.02610492]
1
[ 0.          0.          0.         ... -0.76743238 -0.76712383
 -0.75982732]
[ 6.00758398e-03 -5.12993735e-03  1.34597012e-17 ... -7.67432380e-01
 -7.67123833e-01 -7.59827318e-01]
2
[0. 0. 0. ... 0. 0. 0.]
[-0.10749026 -0.15465772 -0.02345433 ... -0.12317012 -0.10749026
 -0.02288107]
3
[0. 0. 0. ... 0. 0. 0.]
[0.2833834  0.39602153 0.05613337 ... 0.23393438 0.24429604 0.05613152]
4
[0. 0. 0. ... 0. 0. 0.]
[-0.01954368 -0.0826568  -0.10333191 ... -0.01078764 -0.00977184
 -0.00200644]
5
[0. 0. 0. ... 0. 0. 0.]
[-1.95436829e-02 -2.85217375e-02 -3.40489158e-02 ... -6.50714747e-03
 -1.12476843e-20  2.96575373e-04]
6
[0. 0. 0. ... 0. 0. 0.]
[-0.09771841 -0.10241211 -0.09688753 ... -0.1135168  -0.09771841
 -0.02030677]
7
[0. 0. 0. ... 0. 0. 0.]
[ 0.00861104  0.00447296 -0.01752619 ... -0.04108358 -0.01954368
 -0.00291282]
8
[0. 0. 0. ... 0. 0. 0.]
[0.07817473 0.0902334  0.06266835 ... 0

[-0.3023473  -0.43462285 -0.41277624 ... -0.32609463 -0.38463886
 -0.17457073]
100
[0. 0. 0. ... 0. 0. 0.]
[-0.00827671  0.00585754  0.0172464  ... -0.02111395 -0.01804278
 -0.00495013]
101
[0. 0. 0. ... 0. 0. 0.]
[-0.06840289 -0.09070461 -0.07501331 ... -0.05232723 -0.02931552
 -0.00490335]
102
[ 1.93599718  1.93340036  1.93558638 ... -1.87202735 -1.87283814
 -1.87552974]
[ 1.93599718  1.93340036  1.93558638 ... -1.87202735 -1.87283814
 -1.87552974]
103
[0. 0. 0. ... 0. 0. 0.]
[-0.10749026 -0.11855862 -0.08925976 ... -0.10416394 -0.08794657
 -0.01773607]
104
[0. 0. 0. ... 0. 0. 0.]
[-0.05863105 -0.02267029 -0.27594157 ... -0.04356883 -0.04885921
 -0.01199099]
105
[0. 0. 0. ... 0. 0. 0.]
[-0.12703394 -0.14713178 -0.11880532 ... -0.15690022 -0.13680578
 -0.02835192]
106
[0. 0. 0. ... 0. 0. 0.]
[1.28011123 1.29955061 1.121782   ... 1.03356273 1.13353361 0.26112171]
107
[0. 0. 0. ... 0. 0. 0.]
[-0.1172621  -0.11986895 -0.09354299 ... -0.14343713 -0.12703394
 -0.02732282]
108
[0. 0. 0. ...

[0. 0. 0. ... 0. 0. 0.]
[-0.06840289 -0.08244741 -0.06984976 ... -0.10300077 -0.08794657
 -0.01829264]
198
[0. 0. 0. ... 0. 0. 0.]
[-0.09995112 -0.14256889 -0.11013272 ... -0.07009068 -0.11010198
 -0.06066871]
199
[-3.31751419 -3.3554504  -3.39180623 ... -1.15051473 -1.15854957
 -1.16400604]
[-3.31751419 -3.3554504  -3.39180623 ... -1.15051473 -1.15854957
 -1.16400604]
200
[0. 0. 0. ... 0. 0. 0.]
[-0.14657762 -0.06944542  0.01002126 ... -0.18689978 -0.1661213
 -0.03539581]
201
[0. 0. 0. ... 0. 0. 0.]
[-0.07817473 -0.10713779 -0.06176279 ... -0.04355611 -0.04885921
 -0.01209276]
202
[-1.40067507 -1.4010635  -1.40245473 ... -0.13044759 -0.13719665
 -0.14396611]
[-1.40067507 -1.4010635  -1.40245473 ... -0.13044759 -0.13719665
 -0.14396611]
203
[0. 0. 0. ... 0. 0. 0.]
[-0.04885921  0.00810195  0.04393416 ... -0.05212645 -0.04885921
 -0.01077924]
204
[0.3419816  0.36572299 0.3963459  ... 2.1578498  2.16465836 2.16886218]
[0.3419816  0.36572299 0.3963459  ... 2.1578498  2.16465836 2.16886218

[-0.00977184  0.01059936  0.02942196 ... -0.02894743 -0.03908737
 -0.00913764]
313
[-0.2670319  -0.2766812  -0.27439331 ...  1.13204259  1.1280614
  1.11943727]
[-0.2670319  -0.2766812  -0.27439331 ...  1.13204259  1.1280614
  1.11943727]
314
[0. 0. 0. ... 0. 0. 0.]
[-0.14657762 -0.15376933 -0.11067061 ... -0.12621104 -0.1172621
 -0.02562222]
315
[0. 0. 0. ... 0. 0. 0.]
[ 0.00409917 -0.00823823  0.00923817 ... -0.01000795 -0.00977184
 -0.00242182]
316
[0. 0. 0. ... 0. 0. 0.]
[-0.05251966 -0.06526115 -0.05294418 ... -0.0493493  -0.06861739
 -0.0340968 ]
317
[0. 0. 0. ... 0. 0. 0.]
[-0.08794657 -0.09760345 -0.11272208 ... -0.07428164 -0.06840289
 -0.01480524]
318
[0. 0. 0. ... 0. 0. 0.]
[-0.05863105 -0.29722154 -0.40781786 ... -0.05438252 -0.05863105
 -0.01397181]
319
[0. 0. 0. ... 0. 0. 0.]
[-0.18566499 -0.38828992  0.56511152 ... -0.2182623  -0.19543683
 -0.04188864]
320
[0.12169683 0.12095345 0.11433055 ... 0.30453062 0.30488146 0.30448917]
[0.12169683 0.12095345 0.11433055 ... 0.3045

[0. 0. 0. ... 0. 0. 0.]
[-0.25406788 -0.20517644 -0.13647776 ... -0.22415544 -0.23452419
 -0.05438349]
418
[0. 0. 0. ... 0. 0. 0.]
[-0.14657762 -0.06592881 -0.00510862 ... -0.15768211 -0.14657762
 -0.03196796]
419
[-2.73462761 -2.73161627 -2.72106719 ... -2.70473235 -2.70582284
 -2.702146  ]
[-2.73462761 -2.73161627 -2.72106719 ... -2.70473235 -2.70582284
 -2.702146  ]
420
[0. 0. 0. ... 0. 0. 0.]
[-0.21498051 -0.26557021 -0.14347453 ... -0.18057095 -0.17589315
 -0.03967059]
421
[0. 0. 0. ... 0. 0. 0.]
[0.08427034 0.09493221 0.08871866 ... 0.07748614 0.09473842 0.044855  ]
422
[0. 0. 0. ... 0. 0. 0.]
[-0.20520867 -0.22706853 -0.17899155 ... -0.23197058 -0.21498051
 -0.04680177]
423
[0. 0. 0. ... 0. 0. 0.]
[-0.1661213  -0.17805839 -0.15062345 ... -0.16503631 -0.1661213
 -0.03713413]
424
[-2.0464208  -2.04908044 -2.04270567 ... -2.19113807 -2.17990241
 -2.17519791]
[-2.0464208  -2.04908044 -2.04270567 ... -2.19113807 -2.17990241
 -2.17519791]
425
[ 0.00939783  0.02079612  0.03811018 ... -

[-0.92668768 -0.90245574 -0.84604605 ...  0.28345753  0.28963739
  0.28944201]
454
[0. 0. 0. ... 0. 0. 0.]
[-0.05863105 -0.02929082 -0.02470942 ... -0.07815173 -0.07817473
 -0.01719592]
455
[0. 0. 0. ... 0. 0. 0.]
[-0.09771841 -0.11099376 -0.11291256 ... -0.12133674 -0.10749026
 -0.02345516]
456
[0. 0. 0. ... 0. 0. 0.]
[ 0.02931552  0.0085145  -0.00567448 ... -0.00271906  0.00977184
  0.00293899]
457
[0. 0. 0. ... 0. 0. 0.]
[-0.05049876 -0.07162383 -0.0814083  ... -0.05885207 -0.06468795
 -0.02923781]
458
[0. 0. 0. ... 0. 0. 0.]
[-0.03965559 -0.05904542 -0.08755262 ... -0.04224997 -0.04206132
 -0.02013565]
459
[0. 0. 0. ... 0. 0. 0.]
[-0.07817473 -0.08502204 -0.07527482 ... -0.13404513 -0.09771841
 -0.01888668]
460
[0. 0. 0. ... 0. 0. 0.]
[0.15634946 0.19504388 0.11956189 ... 0.12057661 0.1172621  0.02579392]
461
[0. 0. 0. ... 0. 0. 0.]
[-0.15848053 -0.17908531 -0.15118647 ... -0.17501602 -0.20571905
 -0.09251367]
462
[-0.09926487 -0.11167514 -0.12957462 ...  0.13292727  0.13719665
  0

[0.46004814 0.46542228 0.46494423 ... 0.14713769 0.1448187  0.13710646]
543
[0. 0. 0. ... 0. 0. 0.]
[-0.04885921 -0.00703067  0.02151208 ... -0.05466134 -0.04885921
 -0.01000503]
544
[0. 0. 0. ... 0. 0. 0.]
[0.19692109 0.20418128 0.23041662 ... 0.27508522 0.29181321 0.12567216]
545
[0. 0. 0. ... 0. 0. 0.]
[-0.07817473 -0.25079807 -0.32146065 ... -0.09487807 -0.06840289
 -0.01334436]
546
[0. 0. 0. ... 0. 0. 0.]
[-0.22475235 -0.23469218 -0.22066533 ... -0.22680516 -0.21498051
 -0.04680174]
547
[-0.6503338  -0.64674212 -0.64787312 ...  1.33920222  1.34147838
  1.34795294]
[-0.6503338  -0.64674212 -0.64787312 ...  1.33920222  1.34147838
  1.34795294]
548
[3.73807065 4.02082099 4.24547445 ... 0.76533184 0.75458161 0.75335494]
[3.73807065 4.02082099 4.24547445 ... 0.76533184 0.75458161 0.75335494]
549
[0. 0. 0. ... 0. 0. 0.]
[-0.10749026 -0.11474669 -0.06906457 ... -0.10000966 -0.09771841
 -0.02125188]
550
[0. 0. 0. ... 0. 0. 0.]
[ 0.02931552 -0.00582445 -0.03213342 ...  0.00633487  0.019543

[-0.12703394 -0.13520755 -0.15137869 ... -0.09007475 -0.09771841
 -0.02290609]
665
[0. 0. 0. ... 0. 0. 0.]
[-0.14657762 -0.15482971 -0.11425645 ... -0.15460857 -0.14657762
 -0.03130224]
666
[1.19125878 1.19691564 1.20428179 ... 0.64699759 0.64787312 0.64775338]
[1.19125878 1.19691564 1.20428179 ... 0.64699759 0.64787312 0.64775338]
667
[0. 0. 0. ... 0. 0. 0.]
[ 0.01954368  0.0388573  -0.01208724 ...  0.04102593  0.02931552
  0.00506913]
668
[0. 0. 0. ... 0. 0. 0.]
[-0.08794657 -0.08504702 -0.16599024 ... -0.1377016  -0.10749026
 -0.02137718]
669
[0. 0. 0. ... 0. 0. 0.]
[-0.00038671  0.0011398   0.00356619 ... -0.02528716 -0.02417272
 -0.02562449]
670
[-2.58257898 -2.14124624 -1.63873778 ... -1.32632294 -1.30336826
 -1.27246131]
[-2.58257898 -2.14124624 -1.63873778 ... -1.32632294 -1.30336826
 -1.27246131]
671
[ 2.75834559  2.65786971  2.43905169 ... -0.04925026 -0.04573222
 -0.04355896]
[ 2.75834559  2.65786971  2.43905169 ... -0.04925026 -0.04573222
 -0.04355896]
672
[0. 0. 0. ... 0. 

[0.75469311 0.7536983  0.75458161 ... 0.99084958 0.99086474 0.99016275]
790
[-0.28548597 -0.28024446 -0.28201535 ... -0.73125779 -0.67836122
 -0.68012842]
[-0.28548597 -0.28024446 -0.28201535 ... -0.73125779 -0.67836122
 -0.68012842]
791
[0.90141435 0.90796398 0.90702232 ... 0.81641107 0.81555788 0.80705045]
[0.90141435 0.90796398 0.90702232 ... 0.81641107 0.81555788 0.80705045]
792
[0. 0. 0. ... 0. 0. 0.]
[ 0.30292709  0.0341863  -0.13351974 ...  0.32646738  0.31269893
  0.07017937]
793
[-1.42153321 -1.42562713 -1.42532095 ... -1.92650295 -1.92837516
 -1.92707764]
[-1.42153321 -1.42562713 -1.42532095 ... -1.92650295 -1.92837516
 -1.92707764]
794
[0. 0. 0. ... 0. 0. 0.]
[-0.06840289 -0.20519939 -0.24309473 ... -0.04481612 -0.04885921
 -0.01137588]
795
[0. 0. 0. ... 0. 0. 0.]
[-0.03347876 -0.03641526 -0.0194869  ...  0.15859108  0.10504706
 -0.03265932]
796
[-0.73815768 -0.71780114 -0.69360534 ... -0.45369132 -0.46494423
 -0.47125846]
[-0.73815768 -0.71780114 -0.69360534 ... -0.45369132

In [103]:
avgs_of_leads_0 = np.array([np.mean(re.delete_zeros(list_ecg_0lead[k])) for k in range(np.shape(list_ecg_0lead)[0])])
avgs_of_leads_5= np.array([np.mean(re.delete_zeros(list_ecg_5lead[k])) for k in range(np.shape(list_ecg_5lead)[0])])

In [104]:
#print(avgs_of_leads_0)
df

,Unnamed: 0,1dAVb,RBBB,LBBB,SB,AF,ST
0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0
2,2,0,0,0,0,0,0
3,3,0,0,0,0,0,0
4,4,0,0,0,0,0,0
...,...,...,...,...,...,...,...
822,822,0,0,0,0,0,0
823,823,0,0,0,0,0,0
824,824,0,0,0,0,0,0
825,825,0,0,0,0,0,0


In [105]:
series={"moyenne0":avgs_of_leads_0,"moyenne1":avgs_of_leads_1}
pd.DataFrame(series)

,moyenne0,moyenne1
0,0.000694,0.001102
1,-0.169633,0.207946
2,0.002154,0.002939
3,-0.001981,-0.002872
4,0.002119,0.002410
...,...,...
822,0.003140,0.003373
823,0.005104,0.005104
824,0.012788,0.015417
825,0.002781,0.002190


In [106]:
print(series["moyenne0"])

[ 6.93903999e-04 -1.69633079e-01  2.15390494e-03 -1.98050539e-03
  2.11918869e-03  7.43713302e-04 -8.95291922e-04 -8.37502139e-03
 -2.29421445e-03  1.87143147e-02 -8.66002605e-03  4.40976482e-01
 -2.16685413e-03  5.12391062e-04  1.87687069e-03 -1.62606013e-02
  4.76783483e-03  6.06410088e-01  1.44086110e-01  2.22318794e-03
 -4.83318997e-02  4.20779418e-01 -7.76227389e-03  1.54076433e-03
  5.78673689e-03 -2.13103227e-04  1.89642626e-01 -3.90202673e-01
  1.22660076e-01  1.82157061e-01 -1.71492569e-01 -7.63136081e-01
  6.63418967e-02  2.94854324e-03  3.00512916e-03  6.97843609e-04
  3.39859144e-03  4.43038953e-03  1.07331606e+00  3.96528728e-03
 -6.43635267e-01  2.96324189e-01 -6.51618813e-05  4.43892802e-03
  5.14624232e-03  4.06468167e-01 -1.27585613e+00 -1.03534685e-03
  2.87844738e-03  1.29739042e-03  1.78005747e-03 -4.42404938e-01
 -2.13654375e-03  2.94718798e-03  1.51280240e-03  3.24865930e-04
 -1.09171803e-03  6.04953702e-03  8.34675406e-03  9.54783014e-01
 -5.43421116e-01 -2.35713

In [107]:
def delete_zeros(ecg):
    """Delete the first and the last occurences of zero in the ecg"""
    temp = ecg[np.where(np.abs(np.array(ecg)) > 10 ** (-6))[0][0]:]
    print(temp)
    return temp[:np.where(np.abs(np.array(temp)) > 10 ** (-6))[-1][-1]]

a=np.array([0,0,0,1,2,2,-1,0,0])
b=np.array([0,1,3,-2,1,0,0,0])
ab = np.array([a,b])
#delete_zeros(a)